In [10]:
import pandas as pd
import os

current_directory = os.getcwd()
print(f"\nCurrent working directory: {current_directory}\n")

#path_data = "Datasets\covid19-cough-audio-classification"
#filename = os.path.join(path_data, "metadata_compiled.csv")

path_data = "../vs_release_16k"
filename = os.path.join(path_data, "all_meta.csv")

data = pd.read_csv(filename, encoding="latin-1")


Current working directory: c:\Users\kevin\Documents\MMI\Audio Data Science\DataSneeze



In [12]:
data.language.value_counts()

language
EN    2262
PT     277
IT     233
TA     129
HI      94
ES      84
ML      51
TE      35
FR      31
DE      14
ZH      13
AR      12
BN      12
GU       8
RO       8
FA       8
MR       8
RU       7
UR       7
NL       7
VI       6
TR       5
SW       4
KO       4
SQ       4
NE       4
BG       3
PL       3
AF       3
ID       3
ET       3
UK       3
MS       2
LT       2
HY       2
PA       2
MK       2
KA       1
HR       1
HU       1
CA       1
FI       1
SR       1
HE       1
EL       1
SL       1
LV       1
Name: count, dtype: int64

In [13]:
data.head()

,ID,gender,age,country,language,sick
0,m0001,male,36,United States of America,EN,no
1,m0002,male,19,Italy,IT,yes
2,f0003,female,56,United States of America,EN,no
3,f0004,female,41,United States of America,EN,no
4,o0005,other,22,United States of America,EN,no
